In [1]:
# install all the requirements for miniCPM-V
!pip install git+https://github.com/huggingface/transformers
!pip install --upgrade modelscope sentencepiece accelerate bitsandbytes datamodel_code_generator

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-czeb03nq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-czeb03nq
  Resolved https://github.com/huggingface/transformers to commit 8bd2b1e8c23234cd607ca8d63f53c1edfea27462
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9732510 sha256=7457108633944b88edaf68a586c5fff3e089880e5719be60f844608c57589f1c
  Stored in directory: /tmp/pip-ephem-wheel-cache-0a9cll4v/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.0
    Uninstalling transformers-4.44.0:
      Successfully uninstalled transformers-4.44.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install qwen_vl_utils

In [3]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 5.5 MB/s eta 0:00:00a 0:00:01


In [4]:
!git clone https://github.com/modelscope/swift.git
%cd swift
!pip install -e '.[llm]'

Cloning into 'swift'...
remote: Enumerating objects: 20911, done.
remote: Counting objects: 100% (518/518), done.
remote: Compressing objects: 100% (261/261), done.
remote: Total 20911 (delta 314), reused 413 (delta 228), pack-reused 20393 (from 1)
Receiving objects: 100% (20911/20911), 48.64 MiB | 44.47 MiB/s, done.
Resolving deltas: 100% (14871/14871), done.
/kaggle/working/swift
Obtaining file:///kaggle/working/swift
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.1/298.1 kB 8.1 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.5/449.5 kB 24.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 78.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install deepspeed>=0.9.3
!pip install liger-kernel-nightly==0.2.1.dev20240911164559

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 7.8 MB/s eta 0:00:00:00:0100:01


In [6]:
!pip install pyav

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.1/29.1 MB 57.0 MB/s eta 0:00:00:00:0100:01


In [7]:
%pwd

'/kaggle/working/swift'

In [8]:
%cd ..

/kaggle/working


In [9]:
test_split = 'path/to/test/split.csv'

In [10]:
# download images
import os
from time import time
import multiprocessing
import logging
import requests
import pandas as pd
import time as t
import random 

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

img_dir = 'sample_test_image_dir'

def get_sbu_urls():
    df = pd.read_csv(test_split)
    urls = df['image_link'].tolist()
    indices = df.index
    return list(zip(indices, urls))

def scrape_and_save(args):
    url, savepath = args
    retries = 3
    for attempt in range(retries):
        try:
            session = requests.Session()
            response = session.get(url, timeout=1)
            response.raise_for_status()
            with open(savepath, 'wb') as f:
                f.write(response.content)
            # logging.info(f"Successfully downloaded: {url}")
            return True
        except requests.RequestException as e:
            pass
            # logging.error(f"Error downloading {url}: {e} (Attempt {attempt+1}/{retries})")
        except IOError as e:
            # logging.error(f"Error saving file {savepath}: {e}")
            return False
        t.sleep(2)  # small delay before retrying
    return False

if __name__ == '__main__':
    startidx = 0
    chunk_size = 5000  # Break downloads into chunks
    urls = get_sbu_urls()
    total_urls = len(urls)

    if not os.path.exists(img_dir):
        os.makedirs(img_dir)

    starttime = time()
    piccounter = 0

    # Download in chunks
    for chunk_start in range(startidx, total_urls, chunk_size):
        chunk_end = min(chunk_start + chunk_size, total_urls)
        url_chunk = urls[chunk_start:chunk_end]
        
        pool = multiprocessing.Pool(16)  # create a new pool for each chunk

        results = []
        for i, (index, url) in enumerate(url_chunk, start=chunk_start):
            name = f'test_{index}.jpg'
            savepath = os.path.join(img_dir, name)
            result = pool.apply_async(scrape_and_save, ((url, savepath),))
            results.append(result)

        pool.close()  # no more tasks
#         pool.join()  # wait for completion

        # Count successful downloads in the chunk
        successful_downloads = sum([r.get() for r in results])
        piccounter += successful_downloads

        logging.info(f"Downloaded {successful_downloads}/{chunk_size} in chunk {chunk_start//chunk_size + 1}. Total downloaded: {piccounter}/{total_urls}")

        # Throttle requests to prevent server throttling (optional)
        t.sleep(10)  # 10-second break between chunks

    print(f"Downloaded {piccounter}/{total_urls} images.")
    print(f"Total time taken: {time() - starttime:.2f} seconds")

Downloaded 5000/5000 images.
Total time taken: 23.09 seconds


In [11]:
import pandas as pd
import json
import os

entity_unit_map = {
    'depth': ['mm',"'",'cm','centimetre', 'millimetre', 'inch', 'metre', 'foot'],
    'height': ['centimetre', 'millimetre', 'inch', 'metre', 'foot'],
    'width': ['millimetre', 'centimetre', 'inch', 'foot', 'metre'],
    'item_volume': ['cup', 'gallon', 'ounce', 'millilitre', 'cubic foot', 'fluid ounce', 'decilitre', 'cubic inch', 'litre', 'quart', 'pint', 'centilitre'],
    'item_weight': ['gram', 'milligram', 'kilogram', 'kilogram to kilogram', 'ounce', 'pound', 'ton', 'microgram', 'millilitre', 'gigabyte', 'gram to gram', 'kilogram to ilogram', 'candela', 'fluid ounce', 'ounce to ounce', 'pound to pound', 'cup', 'milligram to milligram', 'carat', 'litre', 'e pound', 'nits', 'inch', 'quart', 'watt', 'millimetre', 'centimetre', 'millilitre to millilitre', 'cubic inch'],
    'maximum_weight_recommendation': ['kilogram', 'pound', 'ounce', 'ton', 'milligram', 'kilogram to kilogram', 'gram', 'microgram', 'person', 'pound to pound', 'pound to ound', 'foot', 'kilogram to ilogram', 'horsepower'],
    'voltage': ['volt'],
    'wattage': ['watt', 'horsepower', 'kilowatt', 'kilowatt hour', 'milliampere hour']
}



def prepare_dataset(csv_path, image_dir, output_json_path, question, answer_column):
    # Read the CSV file
    df = pd.read_csv(csv_path)
    # Prepare the data in the required format
    data = []
    for index, row in df.iterrows():
        image_path = os.path.join(image_dir, f"test_{index}.jpg")
        
        # Check if the image file exists
        if not os.path.exists(image_path):
            print(f"Warning: Image file not found for index {index}")
            continue
        

        entry = {
            "system": f"You are an entity extractor OCR model. Given the entity, you can extract the text from the image denoting the entity value. Entity value is always a number followed by its unit like {entity_unit_map[row['entity_name']]}.",
            "query": f"<image>What is the value of the {row['entity_name']} of the item shown? Give me numerical value and unit as seen in the image. Example Input: What is the value of the volume of the item shown, Output : 1 cup. Adhere to instructions.",
            "response": "",
            "images": [f"{image_path}"]
        }
        data.append(entry)
    
    # Write the data to a JSON file
    with open(output_json_path, 'w') as f:
        json.dump(data, f, indent=2)
    
    print(f"Dataset prepared and saved to {output_json_path}")

# Usage
csv_path = test_split
image_dir = '/kaggle/working/sample_test_image_dir/'
output_json_path = '/kaggle/working/sample_test.json'
question = "What is the value of the {row['entity_name']} of the item shown? Give me numerical value as seen in the image. Adhere to instructions."
answer_column = 'entity_value'

prepare_dataset(csv_path, image_dir, output_json_path, question, answer_column)

Dataset prepared and saved to /kaggle/working/sample_test.json


In [12]:
%cp -r /kaggle/input/checkpoint163bothamc/pytorch/default/1 /kaggle/working/

In [13]:
!SIZE_FACTOR=2 MAX_PIXELS=313600 CUDA_VISIBLE_DEVICES=0,1 swift infer \
    --ckpt_dir /kaggle/working/1 \
    --result_dir /kaggle/working/sample_test_results_both/ \
    --load_dataset_config true --merge_lora false \
    --val_dataset /kaggle/working/sample_test.json \
    --quant_method gptq \
    --quantization_bit 4 \
    --max_new_tokens 16

run sh: `/opt/conda/bin/python3.10 /kaggle/working/swift/swift/cli/infer.py --ckpt_dir /kaggle/working/1 --result_dir /kaggle/working/sample_test_results_both/ --load_dataset_config true --merge_lora false --val_dataset /kaggle/working/sample_test.json --quant_method gptq --quantization_bit 4 --max_new_tokens 16`
[INFO:swift] Successfully registered `/kaggle/working/swift/swift/llm/data/dataset_info.json`
[INFO:swift] No vLLM installed, if you are using vLLM, you will get `ImportError: cannot import name 'get_vllm_engine' from 'swift.llm'`
[INFO:swift] No LMDeploy installed, if you are using LMDeploy, you will get `ImportError: cannot import name 'prepare_lmdeploy_engine_template' from 'swift.llm'`
[INFO:swift] Start time of running main: 2024-09-15 12:52:11.811841
[INFO:swift] Using val_dataset, ignoring dataset_test_ratio
[INFO:swift] ckpt_dir: /kaggle/working/1
[INFO:swift] Setting model_info['revision']: master
[INFO:swift] Setting args.eval_human: False
[INFO:swift] args: InferArg

In [14]:
import json
import csv
start = START_IDX
end = END_IDX
jsonl_file_path = 'path/to/output/jsonl'  
csv_file_path = f'/kaggle/working/predictions_{start}_{end}.csv'    

responses = []

def safe_eval(x):
    try:
        # Try to evaluate the expression
        return eval(x)
    except (SyntaxError, NameError, TypeError):
        # If it's not valid Python syntax or raises a name error, skip it
        print(f"Skipping invalid Python syntax: '{x}'")
        return None

with open(jsonl_file_path, 'r') as jsonl_file:
    for line in jsonl_file:
        data = json.loads(line)
        response = data.get('response', '')  
        responses.append(response)

with open(csv_file_path, 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(['index', 'prediction']) 
    for index, response in enumerate(responses):
        if safe_eval(response) is not None:
            response = " ".join(safe_eval(response))
            writer.writerow([index, response])
        else:
            print(index)
            writer.writerow([index, ""])

Skipping invalid Python syntax: '['10000.0', 'billion', 'therm'
1407
Skipping invalid Python syntax: '['10400.0', 'British', 'thermometer'
2429
Skipping invalid Python syntax: '['14300.0', 'billion', 'therm'
2808
